# Correlating Returns

In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## Load API Keys from Environment Variables

In [6]:
# Load .env enviroment variables
load_dotenv("../../../../../../.env")

# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["news_api"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("alpaca_key")
alpaca_secret_key = os.getenv("alpaca_secret")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

## Get AAPL Returns for Past Month

In [12]:
# Set the ticker
ticker = "AAPL"

# Set timeframe to '1D'
timeframe = "1D"

# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
past_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()

# Get 4 weeks worth of historical data for AAPL
df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=past_date,
    end=current_date,
    after=None,
    until=None,
).df

# Display data
df.head()

AAPL                                      
                             open     high       low   close     volume
time                                                                   
2021-11-16 00:00:00-05:00  149.93  151.488  149.3400  151.00   47259035
2021-11-17 00:00:00-05:00  150.99  155.000  150.9900  153.65   79658263
2021-11-18 00:00:00-05:00  153.65  158.670  153.0500  157.89  111869313
2021-11-19 00:00:00-05:00  157.64  161.020  156.5328  160.56  104177063
2021-11-22 00:00:00-05:00  161.68  165.700  161.0000  161.08  104752212

In [13]:
# Drop Outer Table Level
df = df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
df = df.drop(columns=["open", "high", "low", "volume"])

# Since this is daily data, we can keep only the date (remove the time) component of the data
df.index = df.index.date

# Display sample data
df.head()

,close
2021-11-16,151.00
2021-11-17,153.65
2021-11-18,157.89
2021-11-19,160.56
2021-11-22,161.08


In [14]:
# Use the `pct_change` function to calculate daily returns of AAPL
aapl_returns = df.pct_change().dropna()

# Display sample data
aapl_returns.head()

,close
2021-11-17,0.017550
2021-11-18,0.027595
2021-11-19,0.016911
2021-11-22,0.003239
2021-11-23,0.002111


In [15]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > end_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date)[:10],
            to=str(date)[:10],
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates

Note: Be aware that running the 3 requests below will only work once within a 24 hour period due to the request limits imposed by the API provider.

In [20]:
# Get first topic
# YOUR CODE HERE
get_headlines("Apple")

Fetching news about 'Apple'
******************************
retrieving news from: 2021-12-16 00:00:00
retrieving news from: 2021-12-15 00:00:00
retrieving news from: 2021-12-14 00:00:00
retrieving news from: 2021-12-13 00:00:00
retrieving news from: 2021-12-12 00:00:00
retrieving news from: 2021-12-11 00:00:00
retrieving news from: 2021-12-10 00:00:00
retrieving news from: 2021-12-09 00:00:00
retrieving news from: 2021-12-08 00:00:00
retrieving news from: 2021-12-07 00:00:00
retrieving news from: 2021-12-06 00:00:00


NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [18]:
# Get second topic
# YOUR CODE HERE
get_headlines("Facebook")

Fetching news about 'Facebook'
******************************
retrieving news from: 2021-12-16 00:00:00
retrieving news from: 2021-12-15 00:00:00
retrieving news from: 2021-12-14 00:00:00
retrieving news from: 2021-12-13 00:00:00
retrieving news from: 2021-12-12 00:00:00
retrieving news from: 2021-12-11 00:00:00
retrieving news from: 2021-12-10 00:00:00
retrieving news from: 2021-12-09 00:00:00
retrieving news from: 2021-12-08 00:00:00
retrieving news from: 2021-12-07 00:00:00
retrieving news from: 2021-12-06 00:00:00
retrieving news from: 2021-12-05 00:00:00
retrieving news from: 2021-12-04 00:00:00
retrieving news from: 2021-12-03 00:00:00
retrieving news from: 2021-12-02 00:00:00
retrieving news from: 2021-12-01 00:00:00
retrieving news from: 2021-11-30 00:00:00
retrieving news from: 2021-11-29 00:00:00
retrieving news from: 2021-11-28 00:00:00
retrieving news from: 2021-11-27 00:00:00
retrieving news from: 2021-11-26 00:00:00
retrieving news from: 2021-11-25 00:00:00
retrieving new

([['Neymar Jr. cut a streaming deal with Facebook Gaming',
   'Facebook exposes mercenary spy firms that targeted 50000 people - Reuters',
   'Google still ran ads on climate denial, despite promising to stop',
   'EXCLUSIVE Facebook to win conditional EU nod for Kustomer buy, sources say - Reuters',
   'U.S. Cracks Down on Firms Said to Aid China’s Repression of Minorities',
   'The line between online gaming and gambling is fading',
   'Tapping the potential of instant gaming platforms',
   'Meta bans surveillance-for-hire firms for targeting users',
   "Meta says 50,000 people were targeted by 'surveillance for hire' companies",
   'Dutch town set to approve Meta data centre, biggest in Netherlands - Reuters',
   'Kentucky tornadoes: Lost photos found 140 miles away',
   'Ray-Ban Stories update makes the glasses a lot more useful for messaging',
   'Meta Ousts 7 Surveillance-for-Hire Operations From Its Platforms',
   'Are Personalized Ads a Waste of Money?',
   'Does Personalized A

In [19]:
# Get third topic
# YOUR CODE HERE
economy_headlines = get_headlines("economy")

Fetching news about 'economy'
******************************
retrieving news from: 2021-12-16 00:00:00
retrieving news from: 2021-12-15 00:00:00
retrieving news from: 2021-12-14 00:00:00
retrieving news from: 2021-12-13 00:00:00
retrieving news from: 2021-12-12 00:00:00
retrieving news from: 2021-12-11 00:00:00
retrieving news from: 2021-12-10 00:00:00
retrieving news from: 2021-12-09 00:00:00
retrieving news from: 2021-12-08 00:00:00
retrieving news from: 2021-12-07 00:00:00
retrieving news from: 2021-12-06 00:00:00
retrieving news from: 2021-12-05 00:00:00
retrieving news from: 2021-12-04 00:00:00
retrieving news from: 2021-12-03 00:00:00
retrieving news from: 2021-12-02 00:00:00
retrieving news from: 2021-12-01 00:00:00
retrieving news from: 2021-11-30 00:00:00
retrieving news from: 2021-11-29 00:00:00
retrieving news from: 2021-11-28 00:00:00
retrieving news from: 2021-11-27 00:00:00
retrieving news from: 2021-11-26 00:00:00
retrieving news from: 2021-11-25 00:00:00
retrieving news

([['Smartphones Are a New Tax on the Poor',
   'Omicron causing sharp slowdown in Britain’s economy, figures show',
   'Roma rely on festive bouquet cash as Turkey’s economy flounders',
   'Australia jobs surge by record in Nov as economy recovers rapidly - Reuters',
   "Japan's economy seen rebounding in Q4, analysts wary about rising material costs - Reuters",
   'Bank of England raises interest rates to 0.25%',
   'Foreigners turn net buyers of Japanese stocks after three weeks of selling - Reuters',
   'S.Korea c.bank warns against inflation, leaves door open for Jan hike - Reuters',
   'U.S. manufacturing production near three-year high in November - Reuters',
   'Portugal on target for growth, deficit cut despite COVID surge, minister says - Reuters',
   "Long bets on China's yuan at 6-month high; bears pounce on Indian rupee - Reuters",
   'BoE had to raise rates to tackle inflation pressure, Bailey says - Reuters',
   'BoE hikes, Fed pivots, ECB rolls slow as pandemic exits div

In [10]:
# Instantiate SentimentIntensityAnalyzer
# YOUR CODE HERE
sid = SentimentIntensityAnalyzer()

In [11]:
# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment

In [12]:
# Get averages of each topics sentiment
# YOUR CODE HERE
aapl_avg = headline_sentiment_summarizer_avg(apple_headlines)

In [13]:
# Combine Sentiment Averages into DataFrame
# YOUR CODE HERE
topic_sentiments = aapl_return

In [14]:
# Set the index value of the sentiment averages DataFrame to be the series of dates.
# YOUR CODE HERE

In [15]:
# Merge with AAPL returns
# YOUR CODE HERE

# Display data
# YOUR CODE HERE

,close,aapl_avg,trade_avg,economy_avg
2020-12-03,-0.001137,-0.090090,0.155805,0.199570
2020-12-04,-0.005694,0.008565,-0.032140,0.159280
2020-12-07,0.012762,0.126770,0.132200,-0.083920
2020-12-08,0.004281,0.082060,0.187675,0.128210
2020-12-09,-0.021395,0.033005,-0.018410,0.031285
2020-12-10,0.012739,0.145985,0.107165,0.179785
2020-12-11,-0.005924,0.098565,0.047545,0.048705
2020-12-14,-0.005878,0.089605,-0.005755,-0.021345
2020-12-15,0.050259,0.240600,0.185420,0.185560
2020-12-16,-0.001095,0.072350,0.098430,0.086070


In [16]:
# Correlate the headlines' sentiment to returns
# YOUR CODE HERE
topic_sentiments.coor().style.background_gradient()

,close,aapl_avg,trade_avg,economy_avg
close,1,0.547862,0.455142,-0.0524333
aapl_avg,0.547862,1,-0.0109107,-0.425666
trade_avg,0.455142,-0.0109107,1,0.170973
economy_avg,-0.0524333,-0.425666,0.170973,1
